# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission, if necessary. Sections that begin with **'Implementation'** in the header indicate where you should begin your implementation for your project. Note that some sections of implementation are optional, and will be marked with **'Optional'** in the header.

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---

## Step 1: Dataset Exploration

Visualize the German Traffic Signs Dataset. This is open ended, some suggestions include: plotting traffic signs images, plotting the count of each sign, etc. Be creative!


The pickled data is a dictionary with 4 key/value pairs:

- features -> the images pixel values, (width, height, channels)
- labels -> the label of the traffic sign
- sizes -> the original width and height of the image, (width, height)
- coords -> coordinates of a bounding box around the sign in the image, (x1, y1, x2, y2). Based the original image (not the resized version).

In [1]:
# Load pickled data
import os
import pickle
import numpy as np
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from scipy.ndimage.interpolation import rotate
from random import randint

is_labels_encode = False

# TODO: fill this in based on where you saved the training and testing data
training_file = 'traffic-signs-data/train.p'
testing_file = 'traffic-signs-data/test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
# ndarrays
X_data, y_data = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

print("Done loading")

Done loading


# Create Validation Set

In [2]:
# Need to split validation set off first, prior to any data processing.
# Otherwise validation data will leak into the training set during the normalization process.
# Must shuffle training data prior to split!
# Will use 1/5 of current training set for validation set.  
# That would give us a 60/15/25 split which is on the low end for training size, but the autogenerated data will
# augment the training data, which should alleviate the concern of too small training size.

# Get randomized datasets for training and validation
X_train, X_validation, y_train, y_validation = train_test_split(
    X_data,
    y_data,
    test_size=0.2,
    random_state=832289)

print("Validation set created")

Validation set created


# Data Augmentation

In [3]:
### Preprocess the data here.
### Feel free to use as many code cells as needed.

## Create horizontally flipped augmented data
X_flip = X_train[:,:,::-1,:]

## Create randomly rotated augmented data
## TODO: Determine if there's an efficient way to rotate each image by a random angle,
## rather than all images by same random angle.
X_rotate_1 = rotate(X_train, angle=randint(-45,45), axes=(1,2), reshape=False)
X_rotate_2 = rotate(X_train, angle=randint(-45,45), axes=(1,2), reshape=False)
X_rotate_3 = rotate(X_train, angle=randint(-45,45), axes=(1,2), reshape=False)

## TODO: Zoom, Stretching, Color Perbutation

#orig_image = X_train[0]
#rotate_image = X_rotate_1[0]
#plt.imshow(orig_image)
#plt.show()
#plt.imshow(rotate_image)
#plt.show()

X_train = np.vstack((X_train, X_flip, X_rotate_1, X_rotate_2, X_rotate_3))

y_train = np.concatenate((y_train, y_train, y_train, y_train, y_train))
y_train.shape
if(y_train[0] == y_train[31367]):
    print("Copying successful")

Copying successful


# Data Preprocessing

In [4]:
# Convert image to grayscale
def rgb2gray(X):
    return np.tensordot(X, [0.299, 0.587, 0.114], axes=([3],[0]))

# Normalization Method 1: Compute mean/stddev per pixel and channel across the entire batch.  
# Use these values to normalize
def normalize_dataset(data_set):
    return (data_set - np.mean(data_set, axis=0)) / (np.std(data_set, axis=0) + 1e-8)

# Normalization Method 2: Compute mean/stddev per image, normalize each image that way.
def normalize_dataset_alt(data_set):
    return (data_set - np.mean(data_set, axis=(1,2,3), keepdims=True)) / (np.std(data_set, axis=(1,2,3), keepdims=True) + 1e-8)

def normalize_grayscale(X):
    """
    Normalize the image data with Min-Max scaling to a range of [0.1, 0.9]
    :param image_data: The image data to be normalized
    :return: Normalized image data
    """
    a = 0.1
    b = 0.9
    return a + ((X - X.min(axis=0))*(b-a))/(X.max(axis=0) - X.min(axis=0))

# Convert to grayscale
# X_train = rgb2gray(X_train)
# X_validation = rgb2gray(X_validation)
# X_test = rgb2gray(X_test)

# Normalize grayscale intensities
X_train = normalize_dataset_alt(X_train)
X_validation = normalize_dataset_alt(X_validation)
X_test = normalize_dataset_alt(X_test)

# Reshape from (num_samples, 32, 32) -> (num_samples, 32, 32, 1)
# X_train = X_train[:,:,:, None]
# X_validation = X_validation[:,:,:, None]
# X_test = X_test[:,:,:, None]

print(X_train.shape)
print("Done with normalization.")

(156835, 32, 32, 3)
Done with normalization.


# OneHot Encode Labels

In [5]:
from sklearn.preprocessing import LabelBinarizer
if not is_labels_encode:
    # Turn labels into numbers and apply One-Hot Encoding
    encoder = LabelBinarizer()
    encoder.fit(y_train)
    y_train = encoder.transform(y_train)
    y_validation = encoder.transform(y_validation)
    y_test = encoder.transform(y_test)

    # Change to float32, so it can be multiplied against the features in TensorFlow, which are float32
    y_train = y_train.astype(np.float32)
    y_validation = y_validation.astype(np.float32)
    y_test = y_test.astype(np.float32)
    is_labels_encode = True

print('Labels One-Hot Encoded')

Labels One-Hot Encoded


# Save Processed Data

In [6]:
### Save Generated, Processed Data To Local File Cache
### Can't save entire training set due to bug, so splitting training data.
### http://stackoverflow.com/questions/31468117/python-3-can-pickle-handle-byte-objects-larger-than-4gb
directory = "processed_data/"

def saveData(pickle_filename, data):
    if not os.path.isfile(directory + pickle_filename):
        print('Saving data to pickle file...')
        try:
            with open(directory + pickle_filename, 'wb') as pfile:
                pickle.dump(data, pfile, pickle.HIGHEST_PROTOCOL)
        except Exception as e:
            print('Unable to save data to', pickle_file, ':', e)
            raise
    print('Data cached in pickle file.')

for i, X_train_n in enumerate(np.split(X_train, 5)): 
    saveData("X_train" + str(i) + ".pickle", X_train_n)
saveData("y_train.pickle", y_train)
saveData('valid_test.pickle', {'X_validation': X_validation, 'X_test': X_test, 'y_validation': y_validation, 'y_test': y_test})
print("All data cached")

Data cached in pickle file.
Data cached in pickle file.
Data cached in pickle file.
Data cached in pickle file.
Data cached in pickle file.
Data cached in pickle file.
Data cached in pickle file.
All data cached


# Checkpoint
Start from here as augmented, preprocessed data is saved in local file.

In [7]:
import math
import pickle
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tqdm import tqdm

directory = "processed_data/"
# Reload the data
def load_data(filename):
    with open(directory + filename, 'rb') as f:
        return pickle.load(f)

    
X_train = np.vstack([load_data('X_train' + str(i) + '.pickle') for i in range(5)])
y_train = load_data('y_train.pickle')
pickle_data = load_data('valid_test.pickle')
X_validation = pickle_data['X_validation']
y_validation = pickle_data['y_validation']
X_test = pickle_data['X_test']
y_test = pickle_data['y_test']
del pickle_data  # Free up memory

print('Data and modules loaded.')

Data and modules loaded.


# Data Summary

In [8]:
### To start off let's do a basic data summary.

# number of training examples
n_train = len(y_train)

# number of validation examples
n_validation = len(y_validation)

# number of testing examples
n_test = len(y_test)

# what's the shape of an image?
image_shape = X_train[0].shape

# how many classes are in the dataset
n_classes = y_train.shape[1]

print("Number of training examples =", n_train)
print("Number of validation examples =", n_validation)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

Number of training examples = 156835
Number of validation examples = 7842
Number of testing examples = 12630
Image data shape = (32, 32, 3)
Number of classes = 43


----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

There are various aspects to consider when thinking about this problem:

- Your model can be derived from a deep feedforward net or a deep convolutional network.
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

### Implementation

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

# Model Creation

In [11]:
# Input [32 x 32 x 3] - Weight []
# Conv 1a [32 x 32 x 64] - Weight - [3 x 3 x 3 x 64]
# Conv 1b [32 x 32 x 64] - Weight - [3 x 3 x 64 x 64]
# Maxpool [16 x 16 x 64] - Weight - []
# Conv 2a [16 x 16 x 128] - Weight - [3 x 3 x 64 x 128]
# Conv 2b [16 x 16 x 128] - Weight - [3 x 3 x 128 x 128]
# Maxpool [8 x 8 x 128] - Weight - []
# Conv 3a [8 x 8 x 256] - Weight - [3 x 3 x 128 x 256]
# Conv 3b [8 x 8 x 256] - Weight - [3 x 3 x 256 x 256]
# Maxpool [4 x 4 x 256] - Weight - []
# FC 1 [1 x 1 x 512] - Weight - [4 x 4 x 256 x 256]
# Out [1 x 1 x 43] - Weight - [512 x 43]

# Total Weights = 3,263,680

layer_depth = {
    'layer_1': 64,
    'layer_2': 128,
    'layer_3': 256,
    'fully_connected': 512
}

weights = {
    'layer_1a': tf.Variable(tf.truncated_normal([3, 3, 3, layer_depth['layer_1']], stddev=1e-2)),
    'layer_1b': tf.Variable(tf.truncated_normal([3, 3, layer_depth['layer_1'], layer_depth['layer_1']], stddev=1e-2)),
    'layer_2a': tf.Variable(tf.truncated_normal([3, 3, layer_depth['layer_1'], layer_depth['layer_2']], stddev=1e-2)),
    'layer_2b': tf.Variable(tf.truncated_normal([3, 3, layer_depth['layer_2'], layer_depth['layer_2']], stddev=1e-2)),
    'layer_3a': tf.Variable(tf.truncated_normal([3, 3, layer_depth['layer_2'], layer_depth['layer_3']], stddev=1e-2)),
    'layer_3b': tf.Variable(tf.truncated_normal([3, 3, layer_depth['layer_3'], layer_depth['layer_3']], stddev=1e-2)),
    'fully_connected': tf.Variable(tf.truncated_normal([4*4*256, layer_depth['fully_connected']], stddev=1e-2)),
    'out': tf.Variable(tf.truncated_normal([layer_depth['fully_connected'], n_classes], stddev=1e-2))
}

biases = {
    'layer_1a': tf.Variable(tf.zeros(layer_depth['layer_1'])),
    'layer_1b': tf.Variable(tf.zeros(layer_depth['layer_1'])),
    'layer_2a': tf.Variable(tf.zeros(layer_depth['layer_2'])),
    'layer_2b': tf.Variable(tf.zeros(layer_depth['layer_2'])),
    'layer_3a': tf.Variable(tf.zeros(layer_depth['layer_3'])),
    'layer_3b': tf.Variable(tf.zeros(layer_depth['layer_3'])),
    'fully_connected': tf.Variable(tf.zeros(layer_depth['fully_connected'])),
    'out': tf.Variable(tf.zeros(n_classes))
}

def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

# Create model
def conv_net(x, weights, biases):
    # Layer 1 - 32*32*3 to 16*16*64
    conv1a = conv2d(x, weights['layer_1a'], biases['layer_1a'])
    conv1b = conv2d(conv1a, weights['layer_1b'], biases['layer_1b'])
    conv1 = maxpool2d(conv1b)

    # Layer 2 - 16*16*64 to 8*8*128
    conv2a = conv2d(conv1, weights['layer_2a'], biases['layer_2a'])
    conv2b = conv2d(conv2a, weights['layer_2b'], biases['layer_2b'])
    conv2 = maxpool2d(conv2b)

    # Layer 3 - 8*8*128 to 4*4*256
    conv3a = conv2d(conv2, weights['layer_3a'], biases['layer_3a'])
    conv3b = conv2d(conv3a, weights['layer_3b'], biases['layer_3b'])
    conv3 = maxpool2d(conv3b)

    # Fully connected layer - 4*4*256 to 512
    # Reshape conv3 output to fit fully connected layer input
    fc1 = tf.reshape(
        conv3,
        [-1, weights['fully_connected'].get_shape().as_list()[0]])
    fc1 = tf.add(
        tf.matmul(fc1, weights['fully_connected']),
        biases['fully_connected'])
    fc1 = tf.nn.tanh(fc1)

    # Output Layer - class prediction - 512 to 10
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out
    
def get_random_train_batch():
    train_perm = np.arange(500)
    np.random.shuffle(train_perm)
    X_train_acc = X_train[train_perm]
    y_train_acc = y_train[train_perm]
    return X_train_acc, y_train_acc
    
# Parameters
learning_rate = 0.0001
batch_size = 256
training_epochs = 100

# tf Graph input
x = tf.placeholder("float", [None, 32, 32, 3])
y = tf.placeholder("float", [None, n_classes])

logits = conv_net(x, weights, biases)

# Feed dicts for accuracy
valid_feed_dict = {x: X_validation, y: y_validation}
test_feed_dict = {x: X_test, y: y_test}

prediction = tf.nn.softmax(logits)

# Cross entropy
cross_entropy = -tf.reduce_sum(y * tf.log(prediction), reduction_indices=1)

# Training loss
loss = tf.reduce_mean(cross_entropy)

# Accuracy
is_correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct_prediction, tf.float32))

# Define loss and optimizer
optimizer = tf.train.AdamOptimizer (learning_rate=learning_rate).minimize(loss)

# Initializing the variables
init = tf.initialize_all_variables()

In [12]:
# The accuracy measured against the validation set
best_validation_accuracy = 0.0

# Number of iterations of no validation accuracy improvement before stopping early
early_stop_num = 2000

# Last iteration that the validation accuracy improved
last_improvement_iteration = 0

total_iterations = 0

# Measurements use for graphing loss and accuracy
log_batch_step = 50
batches = []
train_acc_batch = []
valid_acc_batch = []

# Create Saver
saver = tf.train.Saver()
save_path = "model/best_model"

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    batch_count = int(math.ceil(n_train/batch_size))
    
    # Training cycle
    for epoch in range(training_epochs):
        # Progress bar
        batches_pbar = tqdm(range(batch_count), desc='Epoch {:>2}/{}'.format(epoch+1, training_epochs), unit='batches')
        
        perm = np.arange(n_train)
        np.random.shuffle(perm)
        X_train = X_train[perm]
        y_train = y_train[perm]
        
        # Loop over all batches
        for batch_i in batches_pbar:
            # Get a batch of training features and labels
            batch_start = batch_i*batch_size
            batch_X = X_train[batch_start : batch_start + batch_size]
            batch_y = y_train[batch_start : batch_start + batch_size]
            
            # Run optimization op (backprop) and cost op (to get loss value)
            _ = sess.run(optimizer, feed_dict={x: batch_X, y: batch_y})
            
            total_iterations += 1
            
            # Log every 100 batches
            if not batch_i % log_batch_step:
                # Calculate Training and Validation accuracy
                X_train_acc, y_train_acc = get_random_train_batch()
                training_accuracy = sess.run(accuracy, feed_dict={x: X_train_acc, y: y_train_acc})
                validation_accuracy = sess.run(accuracy, feed_dict=valid_feed_dict)
                
                # If validation accuracy is an improvement over best seen.
                if validation_accuracy > best_validation_accuracy:
                    
                    # Update best seen validation accuracy
                    best_validation_accuracy = validation_accuracy
                    
                    # Update last iteration to show validation accuracy improvement
                    last_improvement_iteration = total_iterations
                    
                    # Save all variables of the TensorFlow graph to file.
                    saver.save(sess=sess, save_path=save_path)
                    
                    print("Improved Validation Accuracy: ", validation_accuracy)
                
                # Log batches
                previous_batch = batches[-1] if batches else 0
                batches.append(log_batch_step + previous_batch)
                train_acc_batch.append(training_accuracy)
                valid_acc_batch.append(validation_accuracy)
            
            if total_iterations - last_improvement_iteration > early_stop_num:
                print("No improvement found in 1000 batches, stopping optimization.")
                break
            
    print("Optimization Finished!")

acc_plot = plt.subplot(212)
acc_plot.set_title('Accuracy')
acc_plot.plot(batches, train_acc_batch, 'r', label='Training Accuracy')
acc_plot.plot(batches, valid_acc_batch, 'b', label='Validation Accuracy')
acc_plot.set_ylim([0, 1.0])
acc_plot.set_xlim([batches[0], batches[-1]])
acc_plot.legend(loc=4)
plt.tight_layout()
plt.show()
    


Epoch  1/100:   0%|          | 1/613 [01:46<18:09:30, 106.81s/batches]

Improved Validation Accuracy:  0.0544504


Epoch  1/100:   2%|▏         | 11/613 [02:25<1:08:23,  6.82s/batches]

KeyboardInterrupt: 

In [ ]:
    print("Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels}))

### Question 1 

_Describe the techniques used to preprocess the data._

array([[[[ -1.12588178e-01,  -7.43236253e-02,   8.30947465e-03],
         [  6.59364684e-01,   7.60034278e-01,   9.33455810e-01],
         [  1.87969668e+00,   1.89153746e+00,   1.99643028e+00],
         ..., 
         [  2.58519801e-01,   1.59266061e-01,   2.89142971e-01],
         [  1.81419969e-01,   8.21397833e-02,  -2.93219626e-02],
         [  1.57172980e+00,   1.50601174e+00,   1.41619889e+00]],

        [[  1.52166529e-01,   1.51270860e-01,   5.40872298e-01],
         [  8.83012532e-01,   1.03473806e+00,   1.39819051e+00],
         [  1.69592627e+00,   1.77649908e+00,   1.81820126e+00],
         ..., 
         [  1.03462135e+00,   7.73222295e-01,   9.55577396e-01],
         [  3.25599768e-01,   2.22824259e-01,   2.62328998e-03],
         [  5.79562678e-01,   4.06371936e-01,   6.06964534e-01]],

        [[  3.06253153e-01,   2.63630685e-01,   8.34844051e-01],
         [  1.67226934e+00,   1.68663653e+00,   2.00388004e+00],
         [  2.11943211e+00,   2.16282118e+00,   2.032709

**Answer:**

### Generate data additional (if you want to!)
### and split the data into training/validation/testing sets here.
### Feel free to use as many code cells as needed.

Given the data is already centered and boxed, extensive data preprocessing is unlikely to have positive effects. I'll be sticking to standard mean-image subtraction, as well as scaling per channel/pixel to unit variance.

### Question 2

_Describe how you set up the training, validation and testing data for your model. If you generated additional data, why?_

**Answer:**

### Define your architecture here.
### Feel free to use as many code cells as needed.

My plan is to use realtime augmentation of the training data.  Tensorflow has functionality to do this already. I've also read that realtime augmentation can have improved generalization performance compared to pregenerated data, likely because the augmented data is used only once, as opposed to the number of epochs trained.


### Question 3

_What does your final architecture look like? (Type of model, layers, sizes, connectivity, etc.)  For reference on how to build a deep neural network using TensorFlow, see [Deep Neural Network in TensorFlow
](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/b516a270-8600-4f93-a0a3-20dfeabe5da6/concepts/83a3a2a2-a9bd-4b7b-95b0-eb924ab14432) from the classroom._


**Answer:**

In [5]:
### Train your model here.
### Feel free to use as many code cells as needed.

### Question 4

_How did you train your model? (Type of optimizer, batch size, epochs, hyperparameters, etc.)_


**Answer:**

### Question 5


_What approach did you take in coming up with a solution to this problem?_

**Answer:**

---

## Step 3: Test a Model on New Images

Take several pictures of traffic signs that you find on the web or around you (at least five), and run them through your classifier on your computer to produce example results. The classifier might not recognize some local signs but it could prove interesting nonetheless.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Implementation

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [3]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.

### Question 6

_Choose five candidate images of traffic signs and provide them in the report. Are there any particular qualities of the image(s) that might make classification difficult? It would be helpful to plot the images in the notebook._



**Answer:**

In [4]:
### Run the predictions here.
### Feel free to use as many code cells as needed.

### Question 7

_Is your model able to perform equally well on captured pictures when compared to testing on the dataset?_


**Answer:**

In [ ]:
### Visualize the softmax probabilities here.
### Feel free to use as many code cells as needed.

### Question 8

*Use the model's softmax probabilities to visualize the **certainty** of its predictions, [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.11/api_docs/python/nn.html#top_k) could prove helpful here. Which predictions is the model certain of? Uncertain? If the model was incorrect in its initial prediction, does the correct prediction appear in the top k? (k should be 5 at most)*


**Answer:**

### Question 9
_If necessary, provide documentation for how an interface was built for your model to load and classify newly-acquired images._


**Answer:**

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.